# Introdução à Localização de Objetos

A Localização de Objetos (Object Localization) é uma tarefa fundamental em Visão Computacional que estende a tarefa de classificação de imagens. Enquanto a classificação se concentra em determinar a presença de um objeto em uma imagem (o "o quê"), a localização visa não apenas identificar o objeto, mas também determinar sua posição espacial, geralmente por meio de uma caixa delimitadora (bounding box) que o envolve (o "onde"). Este processo é um passo intermediário para tarefas mais complexas, como detecção de múltiplos objetos e segmentação de instâncias. Neste notebook, exploraremos os conceitos fundamentais da localização de objetos, desde a criação de um dataset sintético até a implementação e o treinamento de uma rede neural convolucional (CNN) com múltiplas saídas para prever simultaneamente a classe do objeto e as coordenadas de sua caixa delimitadora.

## Geração do Dataset Sintético

Para treinar nosso modelo, precisamos de um dataset que contenha imagens e suas respectivas anotações. As anotações devem incluir o rótulo da classe do objeto e as coordenadas da sua *bounding box*. Iremos gerar imagens contendo uma de três formas geométricas (retângulo, círculo ou triângulo) em posições, tamanhos e cores aleatórias.

Os arquivos serão salvos em duas pastas: `images/` para os arquivos PNG e `annotations/` para os arquivos JSON correspondentes, que contêm as anotações.

In [ ]:
import os
import json
import math
import random
from PIL import Image, ImageDraw
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torch import nn, optim
import torchvision.models as models
from tqdm.notebook import tqdm

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

### Formatos da Bounding Box

É importante compreender os diferentes formatos de representação de *bounding boxes*. Durante a geração das imagens, utilizaremos o formato `[x0, y0, x1, y1]`, que representa as coordenadas do canto superior esquerdo (top-left) e do canto inferior direito (bottom-right). Este formato é intuitivo para bibliotecas de desenho como a PIL.

No entanto, para o treinamento do modelo, é comum e muitas vezes mais eficaz normalizar as coordenadas e utilizar o formato `[cx, cy, w, h]`, onde:
- `cx`: coordenada x do centro da caixa
- `cy`: coordenada y do centro da caixa
- `w`: largura (width) da caixa
- `h`: altura (height) da caixa

As conversões entre os formatos são dadas por:

$$
cx = \frac{x_0 + x_1}{2} \quad , \quad cy = \frac{y_0 + y_1}{2}
$$

$$
w = x_1 - x_0 \quad , \quad h = y_1 - y_0
$$

Todas as coordenadas `[cx, cy, w, h]` serão normalizadas pela dimensão da imagem para estarem no intervalo [0, 1].

In [ ]:
def convert_to_yolo_format(box, img_size):
    """Converte a bounding box de [x0, y0, x1, y1] para [cx, cy, w, h] normalizado."""
    x0, y0, x1, y1 = box
    img_w, img_h = img_size
    
    dw = 1. / img_w
    dh = 1. / img_h
    
    cx = (x0 + x1) / 2.0
    cy = (y0 + y1) / 2.0
    w = x1 - x0
    h = y1 - y0
    
    cx_norm = cx * dw
    cy_norm = cy * dh
    w_norm = w * dw
    h_norm = h * dh
    
    return [cx_norm, cy_norm, w_norm, h_norm]

def convert_from_yolo_format(box, img_size):
    """Converte a bounding box de [cx, cy, w, h] normalizado para [x0, y0, w, h] em pixels."""
    cx_norm, cy_norm, w_norm, h_norm = box
    img_w, img_h = img_size
    
    w = w_norm * img_w
    h = h_norm * img_h
    x0 = (cx_norm * img_w) - (w / 2)
    y0 = (cy_norm * img_h) - (h / 2)
    
    return [x0, y0, w, h]

In [ ]:
DATASET_DIR = 'data/object_location'
IMG_SIZE = (224, 224)
NUM_IMAGES = 5000
SHAPES = ['circle', 'square', 'triangle']
SIZES = {'small': 40, 'large': 100}  # tamanhos fixos
CLASSES = {shape: i for i, shape in enumerate(SHAPES)}

images_path = os.path.join(DATASET_DIR, 'images')
annotations_path = os.path.join(DATASET_DIR, 'annotations')

os.makedirs(images_path, exist_ok=True)
os.makedirs(annotations_path, exist_ok=True)

for i in range(NUM_IMAGES):
    image = Image.new('RGB', IMG_SIZE, 'black')
    draw = ImageDraw.Draw(image)

    shape_type = random.choice(SHAPES)
    size_label = random.choice(list(SIZES.keys()))
    side = SIZES[size_label]

    color = (
        random.randint(50, 255),
        random.randint(50, 255),
        random.randint(50, 255)
    )

    pos_x0 = random.randint(0, IMG_SIZE[0] - side)
    pos_y0 = random.randint(0, IMG_SIZE[1] - side)
    pos_x1 = pos_x0 + side
    pos_y1 = pos_y0 + side
    box = [pos_x0, pos_y0, pos_x1, pos_y1]

    if shape_type == 'square':
        draw.rectangle(box, fill=color)

    elif shape_type == 'circle':
        draw.ellipse(box, fill=color)

    elif shape_type == 'triangle':
        cx = (pos_x0 + pos_x1) / 2
        cy = (pos_y0 + pos_y1) / 2
        h = (math.sqrt(3) / 2) * side
        p1 = (cx, cy - h / 2)
        p2 = (cx - side / 2, cy + h / 2)
        p3 = (cx + side / 2, cy + h / 2)
        draw.polygon([p1, p2, p3], fill=color)

    img_filename = f'{i:04d}.png'
    image.save(os.path.join(images_path, img_filename))

    yolo_box = convert_to_yolo_format(box, IMG_SIZE)
    annotation = {
        'image': img_filename,
        'label': CLASSES[shape_type],
        'size': size_label,
        'bbox': yolo_box
    }

    ann_filename = f'{i:04d}.json'
    with open(os.path.join(annotations_path, ann_filename), 'w') as f:
        json.dump(annotation, f)

print(f'{NUM_IMAGES} imagens e anotações geradas com sucesso em {DATASET_DIR}.')

## Dataset e DataLoader em PyTorch

Para carregar os dados de forma eficiente no PyTorch, criaremos uma classe `Dataset` customizada. A classe `torch.utils.data.Dataset` é uma classe abstrata que representa um dataset. Uma classe customizada deve implementar três métodos fundamentais:
- `__init__(self, ...)`: Executado uma vez na instanciação do objeto, ideal para inicializar o dataset, como carregar os nomes dos arquivos.
- `__len__(self)`: Deve retornar o tamanho do dataset.
- `__getitem__(self, idx)`: Suporta indexação para obter a i-ésima amostra do dataset. É aqui que carregamos a imagem e sua anotação, aplicamos transformações e retornamos os tensores prontos para o modelo.

In [ ]:
class ShapesDataset(Dataset):
    """Dataset customizado para as formas geométricas."""
    def __init__(self, img_dir, ann_dir, transform=None):
        self.img_dir = img_dir
        self.ann_dir = ann_dir
        self.transform = transform
        # Lista todos os arquivos de imagem, assumindo que eles correspondem às anotações
        self.img_files = sorted([f for f in os.listdir(img_dir) if f.endswith('.png')])

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        # Carrega a imagem
        img_name = self.img_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        
        # Carrega a anotação
        ann_name = img_name.replace('.png', '.json')
        ann_path = os.path.join(self.ann_dir, ann_name)
        with open(ann_path, 'r') as f:
            annotation = json.load(f)
            
        label = torch.tensor(annotation['label'], dtype=torch.long)
        bbox = torch.tensor(annotation['bbox'], dtype=torch.float32)

        # Aplica transformações na imagem, se houver
        if self.transform:
            image = self.transform(image)
            
        return image, (label, bbox)

### Transformações e Preparação dos DataLoaders

Utilizamos `torchvision.transforms` para aplicar transformações comuns em imagens. A transformação `ToTensor()` converte a imagem PIL (intervalo [0, 255]) para um tensor PyTorch (intervalo [0.0, 1.0]) e ajusta a ordem das dimensões (H, W, C) para (C, H, W). A `Normalize()` ajusta os valores dos pixels para terem média e desvio padrão específicos, o que é uma prática padrão ao usar modelos pré-treinados.

Após criar a instância do `Dataset`, dividimos os dados em conjuntos de treino e validação usando `torch.utils.data.random_split`. Finalmente, os `DataLoaders` são criados para gerenciar o carregamento de dados em lotes (*batches*), embaralhando os dados de treino a cada época para reduzir o sobreajuste (*overfitting*).

In [ ]:
# Define a sequência de transformações
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define os caminhos para as pastas
images_path = os.path.join(DATASET_DIR, 'images')
annotations_path = os.path.join(DATASET_DIR, 'annotations')

# Instancia o dataset completo
full_dataset = ShapesDataset(img_dir=images_path, ann_dir=annotations_path, transform=data_transforms)

# Divide em treino e validação (80% treino, 20% validação)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Cria os DataLoaders
BATCH_SIZE = 64
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Tamanho do dataset de treino: {len(train_dataset)}")
print(f"Tamanho do dataset de validação: {len(val_dataset)}")

In [ ]:
for i in range(5):
    img, (label, bbox) = full_dataset[i]

    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = img.permute(1, 2, 0).numpy()
    img = std * img + mean
    img = np.clip(img, 0, 1)
    
    cx, cy, w, h = bbox
    x0 = (cx - w/2) * IMG_SIZE[0]
    y0 = (cy - h/2) * IMG_SIZE[1]
    rect = patches.Rectangle((x0, y0), w*IMG_SIZE[0], h*IMG_SIZE[1], linewidth=2, edgecolor='r', facecolor='none')
    plt.imshow(img)
    plt.gca().add_patch(rect)
    plt.title(SHAPES[label])
    plt.show()

## Arquitetura do Modelo

Para a tarefa de localização, o modelo precisa de duas saídas (cabeças):
1.  **Cabeça de Classificação**: Uma camada linear que recebe os vetores de características (*features*) extraídos pela rede convolucional e produz *logits* para cada classe de objeto.
2.  **Cabeça de Regressão**: Outra camada linear que recebe as mesmas *features* e regride os 4 valores da *bounding box* (`cx`, `cy`, `w`, `h`).

Utilizaremos uma arquitetura de *transfer learning*. Um modelo `MobileNetV2` pré-treinado na ImageNet será usado como *backbone* para extração de características. Congelaremos os pesos do *backbone* e treinaremos apenas as duas novas cabeças que adicionaremos ao final da rede.

In [ ]:
class LocalizationModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # Carrega o backbone pré-treinado
        self.backbone = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
        
        # Remove a camada de classificação original do MobileNetV2
        num_features = self.backbone.classifier[1].in_features
        self.backbone.classifier = nn.Identity()

        # Congela os pesos do backbone
        for param in self.backbone.parameters():
            param.requires_grad = False

        # Descongela as últimas 3 camadas do backbone para fine-tuning
        for param in list(self.backbone.features[-2:].parameters()):
            param.requires_grad = True

        # Cabeça para classificação
        self.classifier_head = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(num_features, num_classes)
        )
        
        # Cabeça para regressão da bounding box
        self.regressor_head = nn.Sequential(
            nn.Linear(num_features, 4),
            nn.Sigmoid() # Garante que as saídas estejam entre 0 e 1
        )

    def forward(self, x):
        features = self.backbone(x)
        features = features.view(features.size(0), -1)
        
        # Saídas das duas cabeças
        class_logits = self.classifier_head(features)
        bbox_pred = self.regressor_head(features)
        
        return class_logits, bbox_pred

## Função de Custo e Treinamento

A função de custo (*loss function*) para este problema de aprendizado multi-tarefa é uma combinação de duas perdas:
- **Perda de Classificação**: `CrossEntropyLoss`, adequada para problemas de classificação multi-classe.
- **Perda de Regressão**: `L1Loss` (ou Mean Absolute Error), que calcula o erro absoluto médio entre as coordenadas preditas e as verdadeiras. A `L1Loss` é geralmente mais robusta a *outliers* do que a `MSELoss` (L2) para regressão de *bounding boxes*.

A perda total é a soma ponderada das duas. Para simplificar, usaremos pesos iguais para ambas.

$$L_{total} = L_{classification} + \lambda L_{regression}$$

Aqui, definiremos $\lambda=1$.

In [ ]:
class LocalizationLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.cross_entropy_loss = nn.CrossEntropyLoss()
        self.l1_loss = nn.L1Loss()

    def forward(self, predictions, targets):
        class_logits, bbox_preds = predictions
        class_labels, bbox_targets = targets
        
        # Calcula as perdas separadamente
        classification_loss = self.cross_entropy_loss(class_logits, class_labels)
        regression_loss = self.l1_loss(bbox_preds, bbox_targets)
        
        # Soma as perdas
        total_loss = classification_loss + regression_loss
        
        return total_loss, classification_loss, regression_loss

### Executando o Treinamento

Agora, vamos instanciar o modelo, a função de custo e o otimizador. Usaremos o otimizador Adam, que é uma escolha robusta para muitas tarefas de deep learning. Note que passamos para o otimizador apenas os parâmetros das novas cabeças (`classifier_head` e `regressor_head`), já que o *backbone* está congelado e não precisa ter seus gradientes atualizados.

In [ ]:
# Instanciação do modelo
num_classes = len(SHAPES)
model = LocalizationModel(num_classes=num_classes).to(device)

# Apenas os parâmetros das novas camadas serão otimizados
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Função de custo
criterion = LocalizationLoss()

In [ ]:
# Dicionário para guardar o histórico
history = {
    'train_total_loss': [], 'train_cls_loss': [], 'train_reg_loss': [], 'train_acc': [],
    'val_total_loss': [], 'val_cls_loss': [], 'val_reg_loss': [], 'val_acc': []
}

num_epochs = 15

# Inicia o ciclo de treinamento
for epoch in range(num_epochs):
    print(f'\nÉpoca {epoch+1}/{num_epochs}')

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
            loader = train_loader
        else:
            model.eval()
            loader = val_loader

        running_total_loss = 0.0
        running_cls_loss = 0.0
        running_reg_loss = 0.0
        running_corrects = 0
        size = len(loader.dataset)

        loop = tqdm(loader, desc=f'{phase.capitalize()}')
        for inputs, (labels, bboxes) in loop:
            inputs = inputs.to(device)
            labels = labels.to(device)
            bboxes = bboxes.to(device)
            
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                class_logits, bbox_preds = model(inputs)
                _, preds = torch.max(class_logits, 1)
                
                loss_total, loss_cls, loss_reg = criterion((class_logits, bbox_preds), (labels, bboxes))
                
                if phase == 'train':
                    loss_total.backward()
                    optimizer.step()

            running_total_loss += loss_total.item() * inputs.size(0)
            running_cls_loss += loss_cls.item() * inputs.size(0)
            running_reg_loss += loss_reg.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels)

        epoch_total_loss = running_total_loss / size
        epoch_cls_loss = running_cls_loss / size
        epoch_reg_loss = running_reg_loss / size
        epoch_acc = running_corrects.double() / size
        
        history[f'{phase}_total_loss'].append(epoch_total_loss)
        history[f'{phase}_cls_loss'].append(epoch_cls_loss)
        history[f'{phase}_reg_loss'].append(epoch_reg_loss)
        history[f'{phase}_acc'].append(epoch_acc.item())

        print(f'{phase.capitalize()} -> Total Loss: {epoch_total_loss:.4f} | Cls Loss: {epoch_cls_loss:.4f} | Reg Loss: {epoch_reg_loss:.4f} | Acc: {epoch_acc:.4f}')

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(18, 10))
fig.suptitle('Histórico de Treinamento', fontsize=16)

# Acurácia
axes[0, 0].plot(history['train_acc'], label='Train Accuracy')
axes[0, 0].plot(history['val_acc'], label='Validation Accuracy')
axes[0, 0].set_title('Acurácia do Modelo')
axes[0, 0].set_xlabel('Época')
axes[0, 0].set_ylabel('Acurácia')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Perda Total
axes[0, 1].plot(history['train_total_loss'], label='Train Total Loss')
axes[0, 1].plot(history['val_total_loss'], label='Validation Total Loss')
axes[0, 1].set_title('Perda Total')
axes[0, 1].set_xlabel('Época')
axes[0, 1].set_ylabel('Perda')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Perda de Classificação
axes[1, 0].plot(history['train_cls_loss'], label='Train Classification Loss')
axes[1, 0].plot(history['val_cls_loss'], label='Validation Classification Loss')
axes[1, 0].set_title('Perda de Classificação (CLS)')
axes[1, 0].set_xlabel('Época')
axes[1, 0].set_ylabel('Perda')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Perda de Regressão
axes[1, 1].plot(history['train_reg_loss'], label='Train Regression Loss')
axes[1, 1].plot(history['val_reg_loss'], label='Validation Regression Loss')
axes[1, 1].set_title('Perda de Regressão (REG)')
axes[1, 1].set_xlabel('Época')
axes[1, 1].set_ylabel('Perda')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

## Inferência e Visualização dos Resultados

Após o treinamento, o passo final é avaliar o desempenho do modelo visualmente. Para quantificar a acurácia da localização, além da inspeção visual, utilizaremos a métrica *Intersection over Union* (IoU).

### Intersection over Union (IoU)

A *Intersection over Union* (IoU), também conhecida como índice de Jaccard, é uma métrica utilizada para avaliar a sobreposição entre duas caixas delimitadoras: a caixa predita ($B_{p}$) e a caixa verdadeira (*ground truth*, $B_{gt}$). O valor do IoU varia de 0 (nenhuma sobreposição) a 1 (sobreposição perfeita).

A fórmula é definida pela razão entre a área da interseção e a área da união das duas caixas:

$$
\text{IoU}(B_p, B_{gt}) = \frac{\text{Área}(B_p \cap B_{gt})}{\text{Área}(B_p \cup B_{gt})}
$$

Onde a área da união pode ser calculada como:

$$
\text{Área}(B_p \cup B_{gt}) = \text{Área}(B_p) + \text{Área}(B_{gt}) - \text{Área}(B_p \cap B_{gt})
$$

Para calcular a área da interseção, determinamos as coordenadas do retângulo de sobreposição. Assumindo que as caixas são definidas por seus cantos superior esquerdo $(x_1, y_1)$ e inferior direito $(x_2, y_2)$, o retângulo de interseção é dado por:

-   $x_{inter\_1} = \max(x_{p1}, x_{gt1})$
-   $y_{inter\_1} = \max(y_{p1}, y_{gt1})$
-   $x_{inter\_2} = \min(x_{p2}, x_{gt2})$
-   $y_{inter\_2} = \min(y_{p2}, y_{gt2})$

A área da interseção é então o produto de sua largura e altura, garantindo que não sejam negativas: $\max(0, x_{inter\_2} - x_{inter\_1}) \times \max(0, y_{inter\_2} - y_{inter\_1})$.

In [ ]:
def calculate_iou(box1, box2):
    """
    Calcula a Intersection over Union (IoU) entre duas bounding boxes.
    As boxes devem estar no formato [x1, y1, x2, y2] (canto superior esquerdo, canto inferior direito).
    """
    # Determina as coordenadas (x, y) do retângulo de interseção
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])

    # Calcula a área da interseção
    # max(0, ...) garante que a área não seja negativa se as caixas não se sobrepuserem
    inter_width = max(0, x2_inter - x1_inter)
    inter_height = max(0, y2_inter - y1_inter)
    intersection_area = inter_width * inter_height

    # Calcula a área de ambas as bounding boxes
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # Calcula a área da união
    union_area = box1_area + box2_area - intersection_area

    # Calcula a IoU
    # Adiciona um pequeno epsilon para evitar divisão por zero
    iou = intersection_area / (union_area + 1e-6)
    
    return iou

In [ ]:
def predict_and_plot(model, dataset, device, num_images=5):
    inv_classes = {v: k for k, v in CLASSES.items()}
    model.eval()
    
    indices = random.sample(range(len(dataset)), num_images)
    
    with torch.no_grad():
        for i in indices:
            image_tensor, (true_label, true_bbox) = dataset[i]
            input_tensor = image_tensor.unsqueeze(0).to(device)
            
            pred_logits, pred_bbox = model(input_tensor)
            
            pred_label_idx = torch.argmax(pred_logits, dim=1).item()
            pred_bbox = pred_bbox[0].cpu().numpy()
            true_bbox = true_bbox.numpy()
            
            # Desnormaliza a imagem para plotagem
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            img = image_tensor.permute(1, 2, 0).numpy()
            img = std * img + mean
            img = np.clip(img, 0, 1)

            # Converte as bounding boxes para formato [x, y, width, height]
            true_bbox_px = convert_from_yolo_format(true_bbox, IMG_SIZE)
            pred_bbox_px = convert_from_yolo_format(pred_bbox, IMG_SIZE)
            
            # Calcula o IoU
            # Converte para [x1, y1, x2, y2] para a função de IoU
            true_box_corners = [true_bbox_px[0], true_bbox_px[1], true_bbox_px[0] + true_bbox_px[2], true_bbox_px[1] + true_bbox_px[3]]
            pred_box_corners = [pred_bbox_px[0], pred_bbox_px[1], pred_bbox_px[0] + pred_bbox_px[2], pred_bbox_px[1] + pred_bbox_px[3]]
            iou_score = calculate_iou(true_box_corners, pred_box_corners)
            
            fig, ax = plt.subplots(1)
            ax.imshow(img)
            ax.axis('off')

            # Cria e adiciona o retângulo da BBox verdadeira (verde)
            rect_true = patches.Rectangle((true_bbox_px[0], true_bbox_px[1]), true_bbox_px[2], true_bbox_px[3], linewidth=2, edgecolor='g', facecolor='none')
            ax.add_patch(rect_true)
            
            # Cria e adiciona o retângulo da BBox predita (vermelho)
            rect_pred = patches.Rectangle((pred_bbox_px[0], pred_bbox_px[1]), pred_bbox_px[2], pred_bbox_px[3], linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect_pred)
            
            true_label_name = inv_classes[true_label.item()]
            pred_label_name = inv_classes[pred_label_idx]
            
            plt.title(f'Verdadeiro: {true_label_name} (Verde)\nPredito: {pred_label_name} (Vermelho)\nIoU: {iou_score:.4f}')
            plt.show()

In [ ]:
# Executa a função de visualização nas imagens do conjunto de validação
predict_and_plot(model, val_dataset.dataset, device, num_images=5)